<a href="https://colab.research.google.com/github/huanfachen/OSRM_playground/blob/main/lecture_note/OSRM_practical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OSRM practical

Author: [Huanfa Chen](github.com/huanfachen)

In [15]:
import datetime
print("Last update:", datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

Last update: 29/06/2022 22:37:35


This practical will achieve the following objectives:

1. Set up and configure the OSRM backend on the server;
2. Use the routing method from the OSRM;
3. Plot the generated route on a map;
4. Compare the routes of driving and cycling.

In [1]:
## install these packages if they are not installed
# !pip install requests
# !pip install folium
# !pip install polyline
# !pip install osrm

In [2]:

import requests
import folium
import polyline

import osrm
from osrm import Point, simple_route


Bad key "text.kerning_factor" on line 4 in
/home/spacetimelab/anaconda3/envs/travel_mode_pred/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


## Set up the OSRM backend

Note: you can skip this step if the OSRM backend has been set up.

The cell below is the bash commands for downloading and processing the data. 

1. Install docker
1. Download the docker image into your computer
1. Download the original OSM data for Greater London. You can replace 'greater-london' with other countries/cities
1. Process and generate the network data. Note that each OSRM backend instance supports only one mode.
1. Start the OSRM server on a selected port (e.g. 5000).

In [3]:
%%bash
## install docker
# sudo apt install docker
## Download the docker image into your computer
# docker pull osrm/osrm-backend
## go to the folder for storing the osm files
# cd your_OSRM_folder
## Download the original OSM data for Greater London. You can replace 'greater-london' with other countries/cities
# wget http://download.geofabrik.de/europe/great-britain/greater-london-latest.osm.pbf
## Process and generate the network data for different mode
# sudo docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-extract -p /opt/car.lua /data/greater-london-latest.osm.pbf
# sudo docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-extract -p /opt/bicycle.lua /data/greater-london-latest.osm.pbf
# sudo docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-extract -p /opt/foot.lua /data/greater-london-latest.osm.pbf

# sudo docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-partition /data/greater-london-latest.osrm
# sudo docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-customize /data/greater-london-latest.osrm
## Start the OSRM server on the port 5000.
# sudo docker run -t -i -p 5000:5000 -v "${PWD}:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/greater-london-latest.osrm

## Connect to the server using a http query

In [4]:
# the URL of the OSRM server for cycling. Use for this practical only and would expire any time
# Replace it with your server if you have one.
url_server_osm_cycle = "http://47.243.162.93:5000"

# Alternatively, you can use the OSM demo server for practice. Note that it only support the mode of 'driving'. The mode provided by the user is neglected
url_server_osm_driving = "http://router.project-osrm.org"

In [5]:
# lat and long of the starting and ending point
lat_start, long_start = 51.522002, -0.136170
lat_end, long_end = 51.500284, -0.124760

We will use a Python package called [osrm](https://github.com/ustroetz/python-osrm) to do the dirty work of formatting the query and interpret the result. This package is called a *wrapper* in some cases.

In [6]:
## install the package of osrm if needed. You can use pip or other methods
## Note that it has some requirements and dependencies
#!pip install osrm

In [7]:
# configure the server. You can use either your own server or a demo server from OSM

# note that in osrm.simple_route, the url should not start with 'http://'
mode = "driving"
if url_server_osm_driving.startswith("http://"):
    url_server_osm_no_htpp = url_server_osm_driving.replace("http://", "")
MyConfig = osrm.RequestConfig(url_server_osm_no_htpp + "/route/v1/" + mode)
print(MyConfig)
result = simple_route(
                    Point(latitude=lat_start, longitude=long_start), Point(latitude=lat_end, longitude=long_end),
                    output='route', geometry='wkt', url_config=MyConfig)
print("The length of this route in metres: ", result[0]['distance'])
print("The duration of this route in seconds: ", result[0]['duration'])

router.project-osrm.org/*/v1/driving
The length of this route in metres:  3227
The duration of this route in seconds:  543.3


**result** is a list, with each element representing a returned geometry (the list of points between start and end points)

In [8]:
# how about on bike?
print("On biking:")
mode = "bike"
if url_server_osm_cycle.startswith("http://"):
    url_server_osm_no_htpp = url_server_osm_cycle.replace("http://", "")
MyConfig = osrm.RequestConfig(url_server_osm_no_htpp + "/route/v1/" + mode)
print(MyConfig)
result = simple_route(
                    Point(latitude=lat_start, longitude=long_start), Point(latitude=lat_end, longitude=long_end),
                    output='route', geometry='wkt', url_config=MyConfig)
print("The length of this route in metres: ", result[0]['distance'])
print("The duration of this route in seconds: ", result[0]['duration'])

On biking:
47.243.162.93:5000/*/v1/bike
The length of this route in metres:  3534.9
The duration of this route in seconds:  1586.1


# The request and response in details

The command above is equivalent to visiting this URL in a browser:
```
http://47.243.162.93:5000/route/v1/driving/-0.136170,51.522002;-0.124760,51.500284?steps=true
```

which is in this format:
```
GET/{service}/{version}/{profile}/{coordinates}[.{format}]?option=value&option=value
```

Below is a snapshot of the response.

![title](../Image/OSRM_response.png)

The response is in JSON format:
```
{"code":"Ok","routes":[{"geometry":"kruh@q`mfNxBu`@nzEkcCfWmoBgs@}cBpoB}jJtkAoDtzCo`BnlKu{@pbKl^|eLmqFn`E}j@~qL_bGsU{vA_k@mDmkBomCtaA{uC`cBwrBsh@w}@~N}XxvBye@viAewBjwFmvApOxd@`fEs@jr@xo@","legs":[{"steps":[],"distance":87717.5,"duration":5342,"summary":"","weight":5342}],"distance":87717.5,"duration":5342,
"weight_name":"routability","weight":5342}],"waypoints":[{"hint":"...... }]}
```

# Plot the route on a map

In [9]:
url = url_server_osm_driving + "/route/v1/driving/-0.136170,51.522002;-0.124760,51.500284?steps=true"
print("URL: ", url)
r = requests.get(url)
res = r.json()
res

URL:  http://router.project-osrm.org/route/v1/driving/-0.136170,51.522002;-0.124760,51.500284?steps=true


{'code': 'Ok',
 'routes': [{'geometry': 'g{myHvrYjBjHdTwTbJ}GcCkd@zBiBtF`ChJdLbGqCdIIxEuE|Gs@fCtAfQcG~R?F{G',
   'legs': [{'steps': [{'geometry': 'g{myHvrYA@EFC@EF',
       'maneuver': {'bearing_after': 325,
        'bearing_before': 0,
        'location': [-0.136277, 51.521956],
        'modifier': 'right',
        'type': 'depart'},
       'mode': 'driving',
       'driving_side': 'right',
       'name': 'Howland Mews East',
       'intersections': [{'out': 0,
         'entry': [True],
         'bearings': [325],
         'location': [-0.136277, 51.521956]}],
       'weight': 7.2,
       'duration': 7.2,
       'distance': 12.5},
      {'geometry': 'y{myHjsYXv@H\\FJd@vA^bABFFP',
       'maneuver': {'bearing_after': 233,
        'bearing_before': 324,
        'location': [-0.136379, 51.522048],
        'modifier': 'left',
        'type': 'turn'},
       'mode': 'driving',
       'driving_side': 'right',
       'name': 'Howland Street',
       'intersections': [{'out': 2,
         'in'

In [10]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat, url=url_server_osm_driving, mode = 'driving'):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    r = requests.get("{}/route/v1/{}/{}".format(url, mode, loc))
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [11]:
test_route = get_route(long_start, lat_start, long_end, lat_end)
test_route

{'route': [(51.52196, -0.13628),
  (51.52142, -0.13778),
  (51.51803, -0.1343),
  (51.51625, -0.13287),
  (51.51691, -0.12689),
  (51.51629, -0.12636),
  (51.51506, -0.12701),
  (51.51325, -0.12912),
  (51.51195, -0.12839),
  (51.51032, -0.12834),
  (51.50923, -0.12727),
  (51.5078, -0.12701),
  (51.50712, -0.12744),
  (51.5042, -0.12614),
  (51.501, -0.12614),
  (51.50096, -0.12472)],
 'start_point': [51.521956, -0.136277],
 'end_point': [51.50096, -0.124724],
 'distance': 3226.9}

In [12]:
def get_map(route):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=13)

    folium.PolyLine(
        route['route'],
        weight=8,
        color='blue',
        opacity=0.6
    ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m

In [13]:
# driving
get_map(get_route(long_start, lat_start, long_end, lat_end, url = url_server_osm_driving, mode = 'driving'))

In [14]:
get_map(get_route(long_start, lat_start, long_end, lat_end, url = url_server_osm_cycle, mode = 'biking'))

## References

1. [Plotting OSRM routes](https://www.thinkdatascience.com/post/2020-03-03-osrm/osrm/)
1. [Explaining request and response](https://medium.com/ivymobility-developers/open-source-routing-machine-43db9ae06fb7)
1. [A tutorial in Chinese and Zhihu](https://zhuanlan.zhihu.com/p/386900671)
1. [OSRM wiki: running multiple profiles on one machine](https://github.com/Project-OSRM/osrm-backend/wiki/Running-OSRM#running-multiple-profiles-on-one-machine)